# Semi-supervised learning - Graph based 

### 1. Harmonic Solution 적용 

1. label 데이터를 기반으로 모델을 학습한다. 
2. 학습된 모델을 기반으로 unlable 데이터의 target value 계산하기
3. label / unlable 데이터를 기반으로 w 행렬을 계산 

> $w_{ij} = exp(\frac{-||x_i - x_j||^2}{\sigma^2})$

4. 최적화 문제 설정하기. label 데이터에 대한 예측은 항상 맞아야 하며, unlable 데이터에 대해선 일부 오차를 허용함. 
- $min_{y \in [0,1]^n} \inf \sum_{i=1}^l (y_i - y_{li})^2 + \sum_{i,j} w_{ij}(y_i - y_j)^2$

> 이때 lable 데이터의 y값은 고정되어 있으니, 손실함수 속 unlable 데이터의 부분을 최소화하는 것을 목표로 함. 

5. harmonic 함수 f를 도입 및 편미분을 통해 Harmonic 함수의 해답을 찾는다. 

> $f(x_i) = \frac{\sum_{j~i} w_{ij} f(x_j)}{\sum_{j~i} w_{ij}}, \forall x_i \in X_u$

> $f_u = -\triangle_{uu}^{-1}\triangle_{ul} y_l$


**Input**
- train_x, train_y, test_x 

**구현해야하는 것**
- model : 데이터의 이진 분류를 하는 함수. sklearn의 SVM을 채택
- w matrix 
- f(x)
- $\triangle$ : D-W 
> D : $\sum_{j=1}^n W_{ij}$ Diagonal degree matrix.  

**Output**
- unlable 데이터에 대한 pred_y


In [14]:
# 라이브러리 설치 및 데이터 설정
import numpy as np 
from sklearn import svm 
from sklearn.datasets import make_blobs

from sklearn.model_selection import train_test_split
X, y = make_blobs(n_samples = 200, centers =2, random_state = 10)

from math import dist


train_x, test_x, train_y, _ = train_test_split(X, y, test_size = 0.9, stratify = y, random_state = 20)


In [46]:
class Graph_SSL(): 
    def __init__(self, train_x, train_y, test_x) : 
        self.train_x = train_x
        self.train_y = train_y 
        self.test_x = test_x
        
        self.n, self.m = np.shape(train_x)
        
        self.model = svm.SVC(kernel = 'linear')
        self.model.fit(train_x, train_y)
        
        self.w_matrix = self.cal_w_matrix() 

    def cal_w_matrix(self) : 
        all_data = np.concatenate((self.train_x, self.test_x), axis =0 ) 
        
        sigma = np.std(all_data)
        w_matrix = np.zeros((len(all_data), len(all_data)))
        
        for i in range(len(all_data)) : 
            for j in range(len(all_data)) : 
                w_matrix[i,j] = np.exp(-dist(all_data[i], all_data[j])**2 / (sigma**2))
        return w_matrix
                
        
    def cal_laplacian(self) : 
        D = np.diag(np.sum(self.w_matrix, axis=1))
        lap = D - self.w_matrix 
        
        return lap[:self.n, :self.n], lap[:self.n, self.n:], lap[self.n:, :self.n], lap[self.n:, self.n:]
        
    def find_harmonic_solution(self) : 
        _, _, lap_ul, lap_uu = self.cal_laplacian() 
        
        label_y = np.reshape(np.array(self.train_y), (-1,1))
        
        pred_y = - np.linalg.inv(lap_uu)@lap_ul@label_y
        
        return pred_y

In [47]:
test = Graph_SSL(train_x, train_y, test_x)

In [48]:
test.find_harmonic_solution()

(180, 180)
(180, 20)
(20, 1)


array([[0.05310698],
       [0.04729144],
       [0.95188648],
       [0.05078255],
       [0.05185938],
       [0.05155637],
       [0.95055245],
       [0.04831832],
       [0.0534692 ],
       [0.05420826],
       [0.95224861],
       [0.048144  ],
       [0.94616067],
       [0.04875709],
       [0.05122873],
       [0.06770707],
       [0.95239598],
       [0.04677833],
       [0.04839573],
       [0.05449296],
       [0.04737542],
       [0.95135939],
       [0.05007368],
       [0.04978069],
       [0.0505704 ],
       [0.95217253],
       [0.94205831],
       [0.05083859],
       [0.94951186],
       [0.94754709],
       [0.95045512],
       [0.05588251],
       [0.94582842],
       [0.95125756],
       [0.06918081],
       [0.04972985],
       [0.0555518 ],
       [0.05224938],
       [0.9528304 ],
       [0.0491875 ],
       [0.95363381],
       [0.05028938],
       [0.95036693],
       [0.95046926],
       [0.95259931],
       [0.95046568],
       [0.0494754 ],
       [0.070

### 2. Harmonic Soluton 개선 - label 데이터의 오차 고려 

1. label 데이터를 기반으로 모델을 학습한다. 
2. 학습된 모델을 기반으로 unlable 데이터의 target value 계산하기
3. label / unlable 데이터를 기반으로 w 행렬을 계산 

> $w_{ij} = exp(\frac{-||x_i - x_j||^2}{\sigma^2})$

4. 최적화 문제 설정하기. label 데이터에 대한 예측은 항상 맞아야 하며, unlable 데이터에 대해선 일부 오차를 허용함. 

> $min_{y \in [0,1]^n} \sum_{i=1}^l (f(x_i) - y_{li})^2 + \lambda * \sum_{i,j} w_{ij}(y_i - y_j)^2$


5. harmonic 함수 f를 도입 및 편미분을 통해 Harmonic 함수의 해답을 찾는다. 

> $f(x_i) = \frac{\sum_{j~i} w_{ij} f(x_j)}{\sum_{j~i} w_{ij}}, \forall x_i \in X_u$

> $f_u =  (I + \lambda \triangle)^{-1} y$


**Input**
- train_x, train_y, test_x 
- $\lambda$

**구현해야하는 것**
- model : 데이터의 이진 분류를 하는 함수. sklearn의 SVM을 채택
- w matrix 
- f(x)
- $\triangle$ : D-W 
> D : $\sum_{j=1}^n W_{ij}$ Diagonal degree matrix.  

**Output**
- unlable 데이터에 대한 pred_y


In [64]:
# label 데이터의 오차를 허용한 개선안

class Graph_SSL(): 
    def __init__(self, train_x, train_y, test_x, lamda) : 
        self.train_x = train_x
        self.train_y = train_y 
        self.test_x = test_x
        self.lamda = lamda
        
        self.n, self.m = np.shape(train_x)
        
        self.model = svm.SVC(kernel = 'linear')
        self.model.fit(train_x, train_y)
        
        self.w_matrix = self.cal_w_matrix() 

    def cal_w_matrix(self) : 
        all_data = np.concatenate((self.train_x, self.test_x), axis =0 ) 
        
        sigma = np.std(all_data)
        w_matrix = np.zeros((len(all_data), len(all_data)))
        
        for i in range(len(all_data)) : 
            for j in range(len(all_data)) : 
                w_matrix[i,j] = np.exp(-dist(all_data[i], all_data[j])**2 / (sigma**2))
        return w_matrix
                
        
    def cal_laplacian(self) : 
        D = np.diag(np.sum(self.w_matrix, axis=1))
        lap = D - self.w_matrix 
        
        return lap
        
    def find_harmonic_solution(self) : 
        lap = self.cal_laplacian()
        
        I = np.diag(np.ones(self.n + len(test_x)))
        y = np.concatenate((np.array(self.train_y), np.zeros(len(self.test_x))), axis =0) 

        label_y = np.reshape(y, (-1,1))

        pred_y = np.linalg.inv(I+self.lamda *lap)@label_y
        
        return pred_y

In [65]:
test = Graph_SSL(train_x, train_y, test_x, 0.5)

In [66]:
test.find_harmonic_solution()

array([[0.10145503],
       [0.101128  ],
       [0.10182912],
       [0.01775984],
       [0.10156844],
       [0.01766013],
       [0.01742286],
       [0.01734664],
       [0.10165734],
       [0.10153957],
       [0.10165523],
       [0.10118354],
       [0.01701624],
       [0.01727431],
       [0.10206544],
       [0.01715538],
       [0.01729364],
       [0.01715717],
       [0.10108891],
       [0.01729129],
       [0.01741036],
       [0.01694664],
       [0.08072258],
       [0.01723156],
       [0.01732574],
       [0.01730321],
       [0.08073286],
       [0.01704897],
       [0.01743934],
       [0.01749005],
       [0.0807832 ],
       [0.01702288],
       [0.08036709],
       [0.01703896],
       [0.01726134],
       [0.01838599],
       [0.08075582],
       [0.01685197],
       [0.01706095],
       [0.0175105 ],
       [0.01695811],
       [0.08071114],
       [0.01719459],
       [0.01717353],
       [0.01722771],
       [0.08081762],
       [0.0800873 ],
       [0.017